# Movie Ratings Lab (Advanced Data Filtering)

### Intro and objectives
#### Apply the concepts learned so far in a real use case

### In this lab you will learn:
1. Implement advanced data filtering

## What I hope you'll get out of this lab
* Gain experience filtering datasets
* Compute basic insigths from data given to you

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

### GroupLens Research provides a number of collections of movie ratings data collected from users of MovieLens in the late 1990s and early 2000s. The data provides movie ratings, movie metadata (genres and year), and demographic data about the users (age, zip code, gender identification, and occupation). Such data is often of interest in the development of recommendation systems based on machine learning algorithms.

### While we do not explore machine learning techniques in detail in this book, You will learn how to slice and dice datasets like these into the exact form you need.

### The MovieLens 1M dataset contains one million ratings collected from six thousand users on four thousand movies. It’s spread across three tables: ratings, user information, and movie information. These files contain 1,000,209 anonymous ratings of approximately 3,900 movies  made by 6,040 MovieLens users who joined MovieLens in 2000.


#### RATINGS FILE DESCRIPTION
================================================================================

All ratings are contained in the file "ratings.dat" and are in the
following format:

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings

#### USERS FILE DESCRIPTION
================================================================================

User information is in the file "users.dat" and is in the following
format:

UserID::Gender::Age::Occupation::Zip-code

All demographic information is provided voluntarily by the users and is
not checked for accuracy.  Only users who have provided some demographic
information are included in this data set.

- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- Occupation is chosen from the following choices:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

#### MOVIES FILE DESCRIPTION
================================================================================

Movie information is in the file "movies.dat" and is in the following
format:

MovieID::Title::Genres

- Titles are identical to titles provided by the IMDB (including
year of release)
- Genres are pipe-separated and are selected from the following genres:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

- Some MovieIDs do not correspond to a movie due to accidental duplicate
entries and/or test entries
- Movies are mostly entered by hand, so errors and inconsistencies may exist

In [2]:
unames = ["user_id", "gender", "age", "occupation", "zip"]
users = pd.read_table("https://raw.githubusercontent.com/thousandoaks/Python4DS-II/main/datasets/users2.txt", sep="::",header=None, names=unames)






<ipython-input-2-5cd50b0e339b>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table("https://raw.githubusercontent.com/thousandoaks/Python4DS-II/main/datasets/users2.txt", sep="::",header=None, names=unames)


In [3]:
rnames = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_table("https://raw.githubusercontent.com/thousandoaks/Python4DS-II/main/datasets/ratings2.txt", sep="::",header=None, names=rnames)



<ipython-input-3-ea5afee8bc36>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table("https://raw.githubusercontent.com/thousandoaks/Python4DS-II/main/datasets/ratings2.txt", sep="::",header=None, names=rnames)


In [4]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("https://raw.githubusercontent.com/thousandoaks/Python4DS-II/main/datasets/movies2.txt", sep="::",header=None, names=mnames)

<ipython-input-4-d13ef0b10199>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table("https://raw.githubusercontent.com/thousandoaks/Python4DS-II/main/datasets/movies2.txt", sep="::",header=None, names=mnames)


In [5]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### Analyzing the data spread across three tables is not a simple task; for example, suppose you wanted to compute mean ratings for a particular movie by gender identity and age. As you will see, this is more convenient to do with all of the data merged together into a single table. Using pandas’s merge function, we first merge ratings with users and then merge that result with the movies data. pandas infers which columns to use as the merge (or join) keys based on overlapping names:

In [8]:
RatingsUsersDataFrame=pd.merge(ratings, users, on='user_id')

In [9]:
RatingsUsersDataFrame

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip
0,1,1193,5,978300760,F,1,10,48067
1,1,661,3,978302109,F,1,10,48067
2,1,914,3,978301968,F,1,10,48067
3,1,3408,4,978300275,F,1,10,48067
4,1,2355,5,978824291,F,1,10,48067
...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106
1000205,6040,1094,5,956704887,M,25,6,11106
1000206,6040,562,5,956704746,M,25,6,11106
1000207,6040,1096,4,956715648,M,25,6,11106


In [10]:
RatingsUsersDataFrameMovies=pd.merge(RatingsUsersDataFrame,movies, on='movie_id')

In [11]:
RatingsUsersDataFrameMovies

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,M,25,6,11106,Sophie's Choice (1982),Drama


In [12]:
## We set movie_id as the new index

RatingsUsersDataFrameMovies.set_index('movie_id',inplace=True)
RatingsUsersDataFrameMovies

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres
movie_id,,,,,,,,,
1193,1,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
661,1,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
914,1,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3408,1,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
2355,1,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...
1091,6040,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1094,6040,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
562,6040,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama


## Let's process the 'genres' column
### We want to extract the first attribute of the 'genres' column and store it as a new colunm called 'genres_main'

In [13]:
RatingsUsersDataFrameMovies.groupby('genres').count()

,user_id,rating,timestamp,gender,age,occupation,zip,title
genres,,,,,,,,
Action,12311,12311,12311,12311,12311,12311,12311,12311
Action|Adventure,10446,10446,10446,10446,10446,10446,10446,10446
Action|Adventure|Animation,345,345,345,345,345,345,345,345
Action|Adventure|Animation|Children's|Fantasy,135,135,135,135,135,135,135,135
Action|Adventure|Animation|Horror|Sci-Fi,618,618,618,618,618,618,618,618
...,...,...,...,...,...,...,...,...
Sci-Fi|Thriller|War,280,280,280,280,280,280,280,280
Sci-Fi|War,1367,1367,1367,1367,1367,1367,1367,1367
Thriller,17851,17851,17851,17851,17851,17851,17851,17851


In [14]:
RatingsUsersDataFrameMovies['genres_main']=RatingsUsersDataFrameMovies['genres'].apply(lambda x:x.split('|')[0])

In [15]:
RatingsUsersDataFrameMovies

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main
movie_id,,,,,,,,,,
1193,1,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,Drama
661,1,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,Animation
914,1,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance,Musical
3408,1,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama,Drama
2355,1,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,Animation
...,...,...,...,...,...,...,...,...,...,...
1091,6040,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy,Comedy
1094,6040,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War,Drama
562,6040,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,Comedy


## Let's select ratings made by female reviewers older than 30

In [16]:
RatingsUsersDataFrameMovies.loc[(RatingsUsersDataFrameMovies['gender']=='F')& (RatingsUsersDataFrameMovies['age']>=30)]



,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main
movie_id,,,,,,,,,,
2406,6,5,978236670,F,50,9,55117,Romancing the Stone (1984),Action|Adventure|Comedy|Romance,Action
1101,6,4,978236670,F,50,9,55117,Top Gun (1986),Action|Romance,Action
3717,6,4,978238371,F,50,9,55117,Gone in 60 Seconds (2000),Action|Crime,Action
1030,6,4,978237691,F,50,9,55117,Pete's Dragon (1977),Adventure|Animation|Children's|Musical,Adventure
1688,6,5,978237570,F,50,9,55117,Anastasia (1997),Animation|Children's|Musical,Animation
...,...,...,...,...,...,...,...,...,...,...
1081,6039,4,956705989,F,45,0,01060,Victor/Victoria (1982),Comedy|Musical,Comedy
1083,6039,3,956706051,F,45,0,01060,"Great Race, The (1965)",Comedy|Musical,Comedy
1086,6039,4,956706182,F,45,0,01060,Dial M for Murder (1954),Mystery|Thriller,Mystery


## Let's select ratings for the movie_id 1688



In [17]:
RatingsUsersDataFrameMovies.loc[1688]

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main
movie_id,,,,,,,,,,
1688,6,5,978237570,F,50,9,55117,Anastasia (1997),Animation|Children's|Musical,Animation
1688,18,4,978154834,F,18,3,95825,Anastasia (1997),Animation|Children's|Musical,Animation
1688,38,3,978046303,F,18,4,02215,Anastasia (1997),Animation|Children's|Musical,Animation
1688,41,5,978039433,F,18,4,15116,Anastasia (1997),Animation|Children's|Musical,Animation
1688,53,3,977979846,M,25,0,96931,Anastasia (1997),Animation|Children's|Musical,Animation
...,...,...,...,...,...,...,...,...,...,...
1688,5930,2,957231548,F,35,17,78681,Anastasia (1997),Animation|Children's|Musical,Animation
1688,5991,3,1000092805,F,35,20,94025,Anastasia (1997),Animation|Children's|Musical,Animation
1688,5995,4,957388219,F,35,1,14618,Anastasia (1997),Animation|Children's|Musical,Animation


## Let's select reviews for movie 1688 with a rating larger than 3.

In [18]:


RatingsUsersDataFrameMovies.loc[(RatingsUsersDataFrameMovies.index == 1688) & (RatingsUsersDataFrameMovies['rating'] > 3)]

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main
movie_id,,,,,,,,,,
1688,6,5,978237570,F,50,9,55117,Anastasia (1997),Animation|Children's|Musical,Animation
1688,18,4,978154834,F,18,3,95825,Anastasia (1997),Animation|Children's|Musical,Animation
1688,41,5,978039433,F,18,4,15116,Anastasia (1997),Animation|Children's|Musical,Animation
1688,75,5,977851476,F,1,10,01748,Anastasia (1997),Animation|Children's|Musical,Animation
1688,92,5,986187240,F,18,4,44243,Anastasia (1997),Animation|Children's|Musical,Animation
...,...,...,...,...,...,...,...,...,...,...
1688,5831,4,957899017,M,25,1,92120,Anastasia (1997),Animation|Children's|Musical,Animation
1688,5851,5,957758333,F,18,20,55410,Anastasia (1997),Animation|Children's|Musical,Animation
1688,5888,4,957481358,M,25,20,64114,Anastasia (1997),Animation|Children's|Musical,Animation


## Let's replace F and M values by Female and Male respectively

In [19]:
RatingsUsersDataFrameMovies.loc[RatingsUsersDataFrameMovies['gender']=='F','gender']='Female'

In [20]:
RatingsUsersDataFrameMovies

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main
movie_id,,,,,,,,,,
1193,1,5,978300760,Female,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,Drama
661,1,3,978302109,Female,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,Animation
914,1,3,978301968,Female,1,10,48067,My Fair Lady (1964),Musical|Romance,Musical
3408,1,4,978300275,Female,1,10,48067,Erin Brockovich (2000),Drama,Drama
2355,1,5,978824291,Female,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,Animation
...,...,...,...,...,...,...,...,...,...,...
1091,6040,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy,Comedy
1094,6040,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War,Drama
562,6040,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,Comedy


In [21]:
RatingsUsersDataFrameMovies.loc[RatingsUsersDataFrameMovies['gender']=='M','gender']='Male'

In [22]:
RatingsUsersDataFrameMovies

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main
movie_id,,,,,,,,,,
1193,1,5,978300760,Female,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,Drama
661,1,3,978302109,Female,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,Animation
914,1,3,978301968,Female,1,10,48067,My Fair Lady (1964),Musical|Romance,Musical
3408,1,4,978300275,Female,1,10,48067,Erin Brockovich (2000),Drama,Drama
2355,1,5,978824291,Female,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,Animation
...,...,...,...,...,...,...,...,...,...,...
1091,6040,1,956716541,Male,25,6,11106,Weekend at Bernie's (1989),Comedy,Comedy
1094,6040,5,956704887,Male,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War,Drama
562,6040,5,956704746,Male,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,Comedy


## Let's select reviews for movies released before 2000


In [23]:
## we need to extract the release year from the title
RatingsUsersDataFrameMovies['release_year']=RatingsUsersDataFrameMovies['title'].apply(lambda x:x[-5:-1])
RatingsUsersDataFrameMovies

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main,release_year
movie_id,,,,,,,,,,,
1193,1,5,978300760,Female,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,Drama,1975
661,1,3,978302109,Female,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,Animation,1996
914,1,3,978301968,Female,1,10,48067,My Fair Lady (1964),Musical|Romance,Musical,1964
3408,1,4,978300275,Female,1,10,48067,Erin Brockovich (2000),Drama,Drama,2000
2355,1,5,978824291,Female,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,Animation,1998
...,...,...,...,...,...,...,...,...,...,...,...
1091,6040,1,956716541,Male,25,6,11106,Weekend at Bernie's (1989),Comedy,Comedy,1989
1094,6040,5,956704887,Male,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War,Drama,1992
562,6040,5,956704746,Male,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,Comedy,1995


In [24]:


# Convert the 'release_year' column to numeric
RatingsUsersDataFrameMovies['release_year_numeric'] = pd.to_numeric(RatingsUsersDataFrameMovies['release_year'])
RatingsUsersDataFrameMovies

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main,release_year,release_year_numeric
movie_id,,,,,,,,,,,,
1193,1,5,978300760,Female,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,Drama,1975,1975
661,1,3,978302109,Female,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,Animation,1996,1996
914,1,3,978301968,Female,1,10,48067,My Fair Lady (1964),Musical|Romance,Musical,1964,1964
3408,1,4,978300275,Female,1,10,48067,Erin Brockovich (2000),Drama,Drama,2000,2000
2355,1,5,978824291,Female,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,Animation,1998,1998
...,...,...,...,...,...,...,...,...,...,...,...,...
1091,6040,1,956716541,Male,25,6,11106,Weekend at Bernie's (1989),Comedy,Comedy,1989,1989
1094,6040,5,956704887,Male,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War,Drama,1992,1992
562,6040,5,956704746,Male,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,Comedy,1995,1995


In [25]:
RatingsUsersDataFrameMovies[RatingsUsersDataFrameMovies['release_year_numeric']<2000]

,user_id,rating,timestamp,gender,age,occupation,zip,title,genres,genres_main,release_year,release_year_numeric
movie_id,,,,,,,,,,,,
1193,1,5,978300760,Female,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,Drama,1975,1975
661,1,3,978302109,Female,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,Animation,1996,1996
914,1,3,978301968,Female,1,10,48067,My Fair Lady (1964),Musical|Romance,Musical,1964,1964
2355,1,5,978824291,Female,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,Animation,1998,1998
1197,1,3,978302268,Female,1,10,48067,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,Action,1987,1987
...,...,...,...,...,...,...,...,...,...,...,...,...
1091,6040,1,956716541,Male,25,6,11106,Weekend at Bernie's (1989),Comedy,Comedy,1989,1989
1094,6040,5,956704887,Male,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War,Drama,1992,1992
562,6040,5,956704746,Male,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,Comedy,1995,1995
